In [ ]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../../quant-container/src')
sys.path.append('../src')
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy
import pickle, cloudpickle

In [ ]:
try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
#if True:
    file_path ='../resources/fx/'
    filenames = ['OIS-N156898469.csv','EUR_JPY_GBP_till_SN.csv','EUR_JPY_GBP_from_SW_till_1Y.csv']
    quote_dict_0 = qc_csv_helper.get_ric_quotes(file_path+filenames[0])
    print(len(quote_dict_0), quote_dict_0.keys())
    quote_dict_1 = qc_csv_helper.get_ric_quotes(file_path+filenames[1])
    print(len(quote_dict_1), quote_dict_1.keys())
    quote_dict_2 = qc_csv_helper.get_ric_quotes(file_path+filenames[2])
    print(len(quote_dict_2), quote_dict_2.keys())
    quote_dict = dict(quote_dict_0,**quote_dict_1)
    quote_dict = dict(quote_dict,**quote_dict_2)
    with open('data.pickle', 'wb') as f:
        pickle.dump(quote_dict, f)

In [ ]:
quote_dict['EURSN='][0].instrument.settle_date

In [ ]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

#spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)
print('*******')
print(my_quote)

In [ ]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [ ]:
# run markouts based on simple instrument ID, with no pricing context
# logging.getLogger().setLevel('INFO')
# logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes[:1000], trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([-10, 0, 10, 100]))\
            > print
        
run(stream)

In [ ]:
# run markouts using PricingContext
# run the markout calc

quotes_eur = quote_dict['EUR=']
quotes_jpy = quote_dict['JPY=']
print(len(quotes_jpy),len(quotes_eur))
quotes_all = op.merge_sorted([quotes_eur[:1000],quotes_jpy[:1000]], lambda x: x.timestamp) > []
             #|\op.map(lambda x: (x.sym, x.timestamp)) > 
run(quotes_all)

my_quote = quotes_all.sink[100]
my_instr_2 = sym_to_instrument()('EURJPY=', my_quote.timestamp.date())
my_instr_2.notionals = (100, -100*124.594511) # just picked a random quote
print(my_instr_2) 
print('*****')
my_trade_2 = FXForwardTrade(my_instr_2, 
                          trade_id = '12346',
                        timestamp = my_quote.timestamp)
print(my_trade_2.timestamp)


In [ ]:
#markouts using PricingContext
trades = [my_trade]
quotes_2 = quotes_all.sink
#print([quote.sym for quote in quotes_2][:100])

prices =  quotes_2 | op.map(FXPricingContextGenerator()) 
#stream = prices | op.map(lambda x: x.timestamp) > print
stream = op.merge_sorted([trades,prices], lambda x:x.timestamp)\
        | op.flat_map(MarkoutCalculator([-10,0, 10, 100])) | op.map(lambda x:str(x))\
        > []
run(stream)       

stream.sink

In [ ]:
# extract just the first quote from every stream
tiny_quote_dict = {key:[quotes[0]] for key, quotes in quote_dict.items()}
print([(key,quotes[0].bid, quotes[0].instrument.ccy, quotes[0].instrument.tenor) for key, quotes in tiny_quote_dict.items()])
with open('../resources/fx/data.pickle', 'wb') as f:
        pickle.dump(tiny_quote_dict, f)


In [ ]:
import datetime
a = datetime.date(2107,1,1)
b = datetime.date(2107,1,2)
from mosaicsmartdata.common.quantlib.bond.fixed_bond import pydate_to_qldate
pydate_to_qldate(b)< pydate_to_qldate(a)


In [ ]:
from datetime import timedelta
from random import shuffle

with open('../resources/fx/data.pickle', 'rb') as f:
        tiny_quote_dict = pickle.load(f)

# that one quote happens to have the wrong date
tiny_quote_dict['USDSWOIS='][0].timestamp -= timedelta(days=1)
tiny_quote_dict['USDSWOIS='][0].instrument.spot_settle_date -= timedelta(days=1)
tiny_quote_dict['USDSWOIS='][0].instrument.maturity_date -= timedelta(days=1)

tiny_quotes_list =[quotes for key, quotes in tiny_quote_dict.items()]

#pc.extra


In [ ]:
shuffle(tiny_quotes_list)

#[print(q[0].instrument.sym, q[0].timestamp.date()) for q in tiny_quotes_list]
stream = op.merge_sorted(tiny_quotes_list, lambda x: x.timestamp) |\
            op.flat_map(FXPricingContextGenerator()) > []
run(stream)
pc = stream.sink[-1]

for sym, quote in tiny_quote_dict.items():
    #if 'OIS' not in sym: #and 'TN' not in sym:
            mid = quote[0].mid
            instr = quote[0].instrument
            est = instr.price(pc)
            if abs(mid-est) > 1e-6:
                print(sym, mid, est, abs(mid-est), 
                      quote[0].timestamp.date(), 
                      instr.settle_date, 
                      instr.maturity_date)

In [ ]:
gbpsn = [q[0] for q in tiny_quotes_list if q[0].sym == 'GBPSN=']
print(gbpsn)

In [ ]:
from mosaicsmartdata.core.curve_utils import construct_OIS_curve, discounting_factor
import datetime

with open('EUR_curve.pickle', 'rb') as f:
        EUR_df = pickle.load(f)
print(EUR_df)
        
eur_curve =  construct_OIS_curve(EUR_df)
a1 = datetime.date(2017, 6, 26)
a2 = datetime.date(2017, 6, 27)
b = datetime.date(2017, 6, 28)
df1 = discounting_factor(eur_curve, a1, b)
df2 = discounting_factor(eur_curve, a2, b)
print(df1, df2)

In [ ]:
pc.extra['EUR']
for sym, quote in tiny_quote_dict.items():
    if 'EUR' in sym:
        print(quote[0].mid, quote[0].instrument.tenor, quote[0].instrument.sym)
print(pc.extra['GBP'])